<a href="https://colab.research.google.com/github/jyothish-mohan/Sports73_App/blob/main/Sports73.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import tensorflow as tf
print(tf.__version__)

'2.5.0'

In [ ]:
train_path = '/sports73/train'
valid_path = '/sports73/valid'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
preprocess_input = tf.keras.applications.inception_v3.preprocess_input

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = preprocess_input)
valid_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_path, target_size = (224,224), batch_size = 64, class_mode = 'categorical')
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size = (224,224), batch_size = 64, class_mode = 'categorical')


Found 10416 images belonging to 73 classes.
Found 365 images belonging to 73 classes.


In [ ]:
base_model = tf.keras.applications.InceptionV3(input_shape = (224,224,3), weights = 'imagenet', include_top = False)

87916544/87910968 [==============================] - 2s 0us/step


In [ ]:
for layer in base_model.layers[:294]:
  layer.trainable = False

In [ ]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out = tf.keras.layers.Dense(73, activation='softmax')(x)

model =  tf.keras.models.Model(inputs = base_model.input, outputs = out)

In [ ]:
for i,layer in enumerate(base_model.layers):
  print(i,layer.name)

0 input_2
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [ ]:
#for layer in model.layers[:113]:
#  layer.trainable = False

In [ ]:
loss = keras.losses.CategoricalCrossentropy()
optim = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optim, loss=loss, metrics=["accuracy"])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=5,
)

epochs = 50
steps_per_epoch = len(train_generator) // 64 # batch_size
validation_steps = len(valid_generator) // 64

history = model.fit(train_generator,epochs=epochs,validation_data=valid_generator,callbacks=[early_stopping])

Epoch 1/50
163/163 [==============================] - 52s 315ms/step - loss: 0.0660 - accuracy: 0.9930 - val_loss: 0.2383 - val_accuracy: 0.9342
Epoch 2/50
163/163 [==============================] - 52s 316ms/step - loss: 0.0315 - accuracy: 0.9985 - val_loss: 0.2376 - val_accuracy: 0.9288
Epoch 3/50
163/163 [==============================] - 52s 316ms/step - loss: 0.0186 - accuracy: 0.9993 - val_loss: 0.2207 - val_accuracy: 0.9315
Epoch 4/50
163/163 [==============================] - 52s 316ms/step - loss: 0.0137 - accuracy: 0.9991 - val_loss: 0.2322 - val_accuracy: 0.9205
Epoch 5/50
163/163 [==============================] - 52s 316ms/step - loss: 0.0123 - accuracy: 0.9989 - val_loss: 0.2146 - val_accuracy: 0.9370
Epoch 6/50
163/163 [==============================] - 52s 315ms/step - loss: 0.0090 - accuracy: 0.9995 - val_loss: 0.2220 - val_accuracy: 0.9342
Epoch 7/50
163/163 [==============================] - 52s 315ms/step - loss: 0.0090 - accuracy: 0.9990 - val_loss: 0.2191 - val_ac

In [ ]:
test_path = "/sports73/test"

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = train_datagen.flow_from_directory(test_path, target_size = (224,224), batch_size = 64, class_mode = 'categorical')

Found 365 images belonging to 73 classes.


In [ ]:
model.evaluate(test_generator)

6/6 [==============================] - 2s 289ms/step - loss: 0.1321 - accuracy: 0.9589


[0.1321355700492859, 0.9589040875434875]

In [ ]:
import os
from PIL import Image
import numpy as np
dir = []
for i in os.listdir(train_path):
  dir.append(i)
labels = dict([(i,j) for i,j in zip([z for z in range(len(dir))],sorted(dir))])


In [ ]:
test = "/sports73/test/boxing/2.jpg"
image = Image.open(test,'r')
img = np.array(image)
print(img.shape)
img = img.reshape(1,224,224,3)
img = preprocess_input(img)

pred = np.argmax(model.predict(img))

(224, 224, 3)


In [ ]:
labels[pred]

'boxing'

In [ ]:
model.save('/Sports73_model.h5')

**Testing after saving the model**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from keras.applications.inception_v3 import preprocess_input

In [ ]:
model = load_model('/Sports73_model.h5')

In [ ]:
import os
from PIL import Image
import numpy as np
dir = []
for i in os.listdir('/sports73/train'):
  dir.append(i)
labels = dict([(i,j) for i,j in zip([z for z in range(len(dir))],sorted(dir))])

In [ ]:
test = "/sports73/test/boxing/2.jpg"
image = Image.open(test,'r')
img = np.array(image)
print(img.shape)
img = img.reshape(1,224,224,3)
img = preprocess_input(img)

pred = np.argmax(model.predict(img))

(224, 224, 3)


In [ ]:
print("The Predicted Sport is {}!".format(labels[pred]))

The Predicted Sport is boxing!


In [ ]:
labels

{0: 'air hockey',
 1: 'ampute football',
 2: 'archery',
 3: 'arm wrestling',
 4: 'balance beam',
 5: 'barell racing',
 6: 'baseball',
 7: 'basketball',
 8: 'billiards',
 9: 'bmx',
 10: 'bobsled',
 11: 'bowling',
 12: 'boxing',
 13: 'bull riding',
 14: 'canoe slamon',
 15: 'cricket',
 16: 'croquet',
 17: 'curling',
 18: 'fencing',
 19: 'field hockey',
 20: 'figure skating men',
 21: 'figure skating pairs',
 22: 'figure skating women',
 23: 'football',
 24: 'formula 1 racing',
 25: 'frisbee',
 26: 'giant slalom',
 27: 'golf',
 28: 'hammer throw',
 29: 'harness racing',
 30: 'high jump',
 31: 'hockey',
 32: 'horse jumping',
 33: 'horse racing',
 34: 'hurdles',
 35: 'ice climbing',
 36: 'jai alai',
 37: 'javelin',
 38: 'judo',
 39: 'lacrosse',
 40: 'luge',
 41: 'motorcycle racing',
 42: 'nascar racing',
 43: 'olympic wrestling',
 44: 'parallel bar',
 45: 'pole vault',
 46: 'polo',
 47: 'pommel horse',
 48: 'rings',
 49: 'rock climbing',
 50: 'rollerblade racing',
 51: 'rowing',
 52: 'rugby